In [84]:
import spacy
import scispacy
import swifter
import pandas as pd
from spacy import displacy
import en_core_sci_sm
import en_core_sci_md
import en_ner_bc5cdr_md
import en_ner_jnlpba_md
import en_ner_craft_md
import en_ner_bionlp13cg_md
from scispacy.abbreviation import AbbreviationDetector
from scispacy.linking import EntityLinker
from collections import OrderedDict,Counter
from pprint import pprint
from tqdm import tqdm
DEBUG = True
tqdm.pandas()

In [85]:
text_file = open("./datasets/covid/phrase_text.txt", 'r')
data = text_file.readlines()
text_file.close()


In [86]:
if DEBUG: 
    print(data[:3])

['sars-cov-2 has been sequenced [ 3 ] .\n', 'a phylogenetic analysis [ 3 , 4 ] found a bat origin for the sars-cov-2 .\n', 'there is a diversity of possible intermediate_hosts for sars-cov-2 , including pangolins , but not mice and rats [ 5 ] .\n']


In [87]:
def display_entities(model,document):
    """ A function that returns a tuple of displacy image of named or unnamed word entities and
        a set of unique entities recognized based on scispacy model in use
        Args: 
            model: A pretrained model from spaCy or ScispaCy
            document: text data to be analysed"""
    nlp = model.load()
    doc = nlp(document)
    displacy_image = displacy.render(doc, jupyter=True,style='ent')
    entity_and_label = set([(X.text, X.label_) for X in doc.ents])
    return  displacy_image, entity_and_label

### Data in one string
---
Since the input data in one string form is too large. We divide it into one sentence each line.

In [88]:
one_string_data = ' '.join(data)

### bc5dr_entities 

In [89]:
len(data)

556143

In [94]:
def get_entity_list(model, data):
    output = []
    nlp = model.load()
    for datum in data:
        doc = nlp(datum)
        entity_and_label = set([(X.text, X.label_) for X in doc.ents])
        output.extend(entity_and_label)
    return output

In [ ]:
bc5dr_entities = get_entity_list(en_ner_bc5cdr_md, data)

In [92]:
bc5dr_entities = display_entities(en_ner_bc5cdr_md, data[1])

<class 'tuple'>


In [93]:
bc5dr_entities

(None, {('3 , 4 ] found a bat origin for the sars-cov-2', 'CHEMICAL')})

In [4]:
nlp = en_ner_bc5cdr_md.load()

In [59]:
doc = nlp("cd4i")

In [60]:
for d in doc:
    print(d)

cd4i


In [61]:
for i in range(len(doc)):
#     if (doc[i].tag_ == 'NNS'):
    print(doc[i].tag_, ", ", doc[i])

NN ,  cd4i


In [62]:
doc.ents

()

In [58]:
displacy_image = displacy.render(doc, jupyter=True,style='ent')

In [20]:
bc5dr_entities = display_entities(en_ner_bc5cdr_md, data[2])

<class 'tuple'>


intermediate_hosts
CHEMICAL


In [8]:
bc5dr_entities = display_entities(en_ner_bc5cdr_md, data[1])

In [33]:
doc

there is a diversity of possible intermediate_hosts for sars-cov-2 , including pangolins , but not mice and rats [ 5 ] .

In [34]:
bc5dr_entities[1]

{('intermediate_hosts', 'CHEMICAL'),
 ('mice', 'WILDLIFE'),
 ('pangolins', 'WILDLIFE'),
 ('rats', 'WILDLIFE'),
 ('sars-cov-2', 'COVID')}